# Problem 1a)
### ADD Algorithm

In [ ]:
import numpy as np

nodes = ['A','B','C','D','E','F']
f0 = [40, 100, 50, 115, 70, 100] # fixed costs
f=f0.copy()
h = [8, 10, 6, 5, 9, 10] # demand


c = {(0,0): 0 , (0,1): 10, (0,2): 16, (0,3): 10, (0,4): 12, (0,5): 16,
     (1,0): 10, (1,1): 0 , (1,2): 6 , (1,3): 7 , (1,4): 16, (1,5): 13,
     (2,0): 16, (2,1): 6 , (2,2): 0 , (2,3): 7 , (2,4): 16, (2,5): 9 ,
     (3,0): 10, (3,1): 7 , (3,2): 7 , (3,3): 0 , (3,4): 9 , (3,5): 6 ,
     (4,0): 12, (4,1): 16, (4,2): 16, (4,3): 9 , (4,4): 0 , (4,5): 9 ,
     (5,0): 16, (5,1): 13, (5,2): 9 , (5,3): 6 , (5,4): 9 , (5,5): 0 } # transportation distances * cost per unit demand (=1)


weighted_demand = np.zeros((6,6))
for i in range(6):
	for j in range(6):
		weighted_demand[i, j] = c[(i, j)] * h[i]
print(weighted_demand)

totals = [sum(weighted_demand[:,i]) * 1 + f[i] for i in range(6)]
print(totals)

[[  0.  80. 128.  80.  96. 128.]
 [100.   0.  60.  70. 160. 130.]
 [ 96.  36.   0.  42.  96.  54.]
 [ 50.  35.  35.   0.  45.  30.]
 [108. 144. 144.  81.   0.  81.]
 [160. 130.  90.  60.  90.   0.]]
[np.float64(554.0), np.float64(525.0), np.float64(507.0), np.float64(448.0), np.float64(557.0), np.float64(523.0)]


In [ ]:
facilities_to_open = [nodes[np.argmin(totals)]]
print("Optimal location is at location:", nodes[np.argmin(totals)], "with cost:", min(totals))

Optimal location is at location: D with cost: 448.0


In [ ]:
weighted_demand[np.argmin(totals),:] = 0

## Update all distances that are closer to the new facility than node i
weighted_demand[0,2] = h[0]*c[(0,3)]
weighted_demand[0,4] = h[0]*c[(0,3)]
weighted_demand[0,5] = h[0]*c[(0,3)]
weighted_demand[1,0] = h[1]*c[(1,3)]
weighted_demand[1,4] = h[1]*c[(1,3)]
weighted_demand[1,5] = h[1]*c[(1,3)]
weighted_demand[2,0] = h[2]*c[(2,3)]
weighted_demand[2,4] = h[2]*c[(2,3)]
weighted_demand[2,5] = h[2]*c[(2,3)]
weighted_demand[4,0] = h[4]*c[(4,3)]
weighted_demand[4,1] = h[4]*c[(4,3)]
weighted_demand[4,2] = h[4]*c[(4,3)]
weighted_demand[5,0] = h[5]*c[(5,3)]
weighted_demand[5,1] = h[5]*c[(5,3)]
weighted_demand[5,2] = h[5]*c[(5,3)]
weighted_demand[5,4] = h[5]*c[(5,3)]
print(weighted_demand)
for i in range(6):
    if nodes[i] == facilities_to_open[-1]:
        continue
    f[i] += f[np.argmin(totals)]
print(f)
totals = [sum(weighted_demand[:,i]) * 1 + f[i] for i in range(6)]
print(totals)

[[ 0. 80. 80. 80. 80. 80.]
 [70.  0. 60. 70. 70. 70.]
 [42. 36.  0. 42. 42. 42.]
 [ 0.  0.  0.  0.  0.  0.]
 [81. 81. 81. 81.  0. 81.]
 [60. 60. 60. 60. 60.  0.]]
[155, 215, 165, 115, 185, 215]
[np.float64(408.0), np.float64(472.0), np.float64(446.0), np.float64(448.0), np.float64(437.0), np.float64(488.0)]


In [ ]:
facilities_to_open.append(nodes[np.argmin(totals)])
print("Next optimal location is at location:", nodes[np.argmin(totals)], "with cost:", min(totals))

Next optimal location is at location: A with cost: 408.0


In [ ]:
weighted_demand[np.argmin(totals),:] = 0
## No distances to update this step
print(weighted_demand)
for i in range(6):
    if nodes[i] == facilities_to_open[-1]:
        continue
    f[i] += f[np.argmin(totals)]
print(f)
totals = [sum(weighted_demand[:,i]) * 1 + f[i] for i in range(6)]
print(totals)

[[ 0.  0.  0.  0.  0.  0.]
 [70.  0. 60. 70. 70. 70.]
 [42. 36.  0. 42. 42. 42.]
 [ 0.  0.  0.  0.  0.  0.]
 [81. 81. 81. 81.  0. 81.]
 [60. 60. 60. 60. 60.  0.]]
[155, 370, 320, 270, 340, 370]
[np.float64(408.0), np.float64(547.0), np.float64(521.0), np.float64(523.0), np.float64(512.0), np.float64(563.0)]


Now that we have no new facilities that give us a better objective value, we stop. So our optimal facility locations are

In [ ]:
print(facilities_to_open)

['D', 'A']


Locations D and A, with a total cost of \$408. The total fixed facility location costs is \$115 + \$40 = \$155, which means the total transportation cost is \$408 - \$155 = \$253.

## Problem 1b)
### IP Solver

In [ ]:
from gurobipy import Model, GRB, quicksum

I = [i for i in range(6)]
J = [j for j in range(6)]
IJ = [(i,j) for i in I for j in J]


model = Model("UFLP_1b")
x = model.addVars(J, vtype=GRB.BINARY)
y = model.addVars(IJ, vtype=GRB.CONTINUOUS, lb=0)

model.setObjective(quicksum(f0[j]*x[j] for j in J) + quicksum(h[i]*c[i,j]*y[i,j] for (i,j) in IJ), GRB.MINIMIZE)
model.addConstrs((quicksum(y[i,j] for j in J) == 1 for i in I))
model.addConstrs((y[i,j] <= x[j] for (i,j) in IJ))
model.update()
model.optimize()

Set parameter Username
Academic license - for non-commercial use only - expires 2026-09-20
Gurobi Optimizer version 12.0.3 build v12.0.3rc0 (win64 - Windows 11.0 (26100.2))

CPU model: 13th Gen Intel(R) Core(TM) i7-13700, instruction set [SSE2|AVX|AVX2]
Thread count: 16 physical cores, 24 logical processors, using up to 24 threads

Optimize a model with 42 rows, 42 columns and 108 nonzeros
Model fingerprint: 0x47c0ba55
Variable types: 36 continuous, 6 integer (6 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [3e+01, 2e+02]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+00]
Presolve time: 0.00s
Presolved: 42 rows, 42 columns, 108 nonzeros
Variable types: 36 continuous, 6 integer (6 binary)
Found heuristic solution: objective 475.0000000
Found heuristic solution: objective 435.0000000

Root relaxation: objective 3.450000e+02, 11 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      

In [ ]:
x

#y

{0: <gurobi.Var C0 (value 1.0)>,
 1: <gurobi.Var C1 (value -0.0)>,
 2: <gurobi.Var C2 (value 1.0)>,
 3: <gurobi.Var C3 (value -0.0)>,
 4: <gurobi.Var C4 (value 1.0)>,
 5: <gurobi.Var C5 (value -0.0)>}

The IP Solver finds a solution of opening facilities A, C, and E, with an objective cost of \$345. This gives a total fixed facility cost of \$160, and a total transportation cost of \$185.

## Problem 1c)
### Lagrangian Relaxation

In [ ]:
def lr_ucflp(f0, c, h, I, J, max_iter=50):
    avg_demand = np.mean(h)
    avg_fixed = np.mean(f0)

    lam_vals = {i: 10*(avg_demand + avg_fixed) for i in I}

    alpha = 1.0

    for iter in range(max_iter):
        print("--- Iteration {} ---".format(iter))

        Y = {(i,j): 0 for i in I for j in J}
        X = {}
        V = {}


        for j in J:
            V[j] = f0[j] + sum(min(0, alpha * h[i] * c[(i,j)] - lam_vals[i]) for i in I)
            X[j] = 1 if V[j] < 0 else 0
            for i in I:
                Y[(i,j)] = 1 if (X[j] == 1 and alpha * h[i] * c[(i,j)] - lam_vals[i] < 0) else 0


        l_bound = sum(f0[j]*X[j] for j in J) + sum((alpha * h[i] * c[(i,j)] - lam_vals[i]) * Y[(i,j)] for i in I for j in J) + sum(lam_vals[i] for i in I)
        print("LB:", l_bound)

        # Claude helped me with this part
        ##########################
        open_facilities = [j for j in J if X[j] == 1]
        if len(open_facilities) == 0:
             min_vj = min(V.values())
             best_facility = [j for j in J if V[j] == min_vj][0]
             X[best_facility] = 1
             open_facilities = [best_facility]

        Y_feasible = {(i,j): 0 for i in I for j in J}
        for i in I:
             best_cost = float('inf')
             best_facility = None
             for j in open_facilities:
                cost = h[i] * c[(i,j)]
                if cost < best_cost:
                    best_cost = cost
                    best_facility = j
             if best_facility is not None:
                Y_feasible[(i, best_facility)] = 1
        ##########################

        u_bound = sum(f0[j] * X[j] for j in J) + alpha * sum(h[i] * c[(i,j)] * Y_feasible[(i,j)] for i in I for j in J)
        print("UB:", u_bound)

        violations_sq = 0
        violations = {}
        for i in I:
            violation = 1 - sum(Y[(i,j)] for j in J)
            violations[i] = violation
            violations_sq += violation**2

        if violations_sq > 0:
            step_size = (u_bound - l_bound) / violations_sq
        else:
            step_size = 0
            print("No constraints violated, optimal solution found.")
            break

        print("Step size:", step_size)

        for i in I:
            lam_vals[i] = max(0, lam_vals[i] + step_size * violations[i])

        print("Updated lambdas:", {i: f"{lam_vals[i]:.2f}" for i in I})

        if u_bound - l_bound < 1:
            print("Converged from gap < 1.")
            break

    return {
    'X': X,
    'Y_relaxed': Y,
    'Y_feasible': Y_feasible,
    'lower_bound': l_bound,
    'upper_bound': u_bound,
    'gap': u_bound - l_bound,
    'lambdas': lam_vals
    }



result = lr_ucflp(f0, c, h, I, J, max_iter=50)

print(f"\n=== FINAL RESULTS ===")
print(f"Lower Bound: {result['lower_bound']:.2f}")
print(f"Upper Bound: {result['upper_bound']:.2f}")
print(f"Gap: {result['gap']:.2f}")
print("Final facility locations:", [j for j in J if result['X'][j] == 1])
print("Final feasible assignments:")
for i in I:
    for j in J:
        if result['Y_feasible'][(i,j)] == 1:
            print(f"  Customer {i} -> Facility {j}")

--- Iteration 0 ---
LB: -23036.000000000015
UB: 475.0
Step size: 156.7400000000001
Updated lambdas: {0: '87.97', 1: '87.97', 2: '87.97', 3: '87.97', 4: '87.97', 5: '87.97'}
--- Iteration 1 ---
LB: -5.4999999999936335
UB: 475.0
Step size: 10.223404255319014
Updated lambdas: {0: '67.52', 1: '67.52', 2: '57.30', 3: '36.85', 4: '67.52', 5: '77.74'}
--- Iteration 2 ---
LB: 330.2631205673755
UB: 383.0
Step size: 26.368439716312253
Updated lambdas: {0: '67.52', 1: '67.52', 2: '57.30', 3: '36.85', 4: '93.89', 5: '104.11'}
--- Iteration 3 ---
LB: 303.74219858156016
UB: 350.0
Step size: 6.608257345491405
Updated lambdas: {0: '67.52', 1: '67.52', 2: '50.69', 3: '30.24', 4: '87.28', 5: '90.90'}
--- Iteration 4 ---
LB: 339.34620060790303
UB: 345.0
Step size: 2.8268996960484856
Updated lambdas: {0: '67.52', 1: '67.52', 2: '50.69', 3: '33.07', 4: '87.28', 5: '88.07'}
--- Iteration 5 ---
LB: 341.1365754812564
UB: 345.0
Step size: 1.9317122593718068
Updated lambdas: {0: '67.52', 1: '67.52', 2: '50.69',

The Lagrangian relaxation solver has found the same solution as the IP solver, with facilities at locations A, C, and E, with an objective cost of \$345. Same fixed (\$160) and transportation (\$185) costs as the IP.

# Problem 2a)
$$\max \sum_{i \in I} h_i z_i$$
$$s.t. \ \  z_i \leq \sum_{j \in J} a_{ij} x_j, \forall i \in I$$
$$\sum_{j \in J} x_j = P$$
$$x_j \in \{0,1\} \  \forall j \in J, \ \ \  z_i \in \{0,1\} \  \forall i \in I$$

where $h_i$ is the demand at node $i$, $Z_i=1$ if node $i \in I$ is covered, $0$ if not, $a_{ij} = 1$ if candidate site $j$ can cover demand at node $i \in I$ and $0$ if not, and $X_j=1$ if we locate a facility at candidate site $j$, $0$ if not.

Problem 2b) ADD Algorithm

| Candidate Site | Nodes Covered | Demand Covered |
|---------------|---------------|----------------|
| A             | A, D, E       | 39             |
| B             | B, F          | 47             |
| C             | C             | 26             |
| D             | D, A          | 28             |
| E             | E, A          | 29             |
| F             | F, B, I       | 82             |
| G             | G             | 14             |
| H             | H, I          | 66             |
| I             | I, F, H, J    | 116            |
| J             | J, I          | 65             |

The optimal facility location for $P=1$ is at location, $I$, covering demand nodes $I, F, H, J$, with total covered demand of 116.

Problem 2c) ADD Algorithm

| Candidate Site | Previously Uncovered Nodes Covered | Previously Uncovered Demand Covered |
|---------------|-----------------------------------|-------------------------------------|
| A             | A, D, E                           | 39                                  |
| B             | B                                 | 27                                  |
| C             | C                                 | 26                                  |
| D             | D, A                              | 28                                  |
| E             | E, A                              | 29                                  |
| F             | -                                 | -                                   |
| G             | G                                 | 14                                  |
| H             | -                                 | -                                   |
| I             | -                                 | -                                   |
| J             | -                                 | -                                   |

For $P=2$, in addition to facility $I$, it is best to locate the second facility at location $A$. Between the two facilities, this will cover nodes $A, D, E, F, I, H, J$, covering a total demand of 155.

## Problem 2b)
### IP Solver

In [ ]:
def MCLP(h, p, A):
    model = Model('MCLP_1')
    x = model.addVars(J2, vtype=GRB.BINARY)
    z = model.addVars(I2, vtype=GRB.BINARY)

    model.setObjective(quicksum(h2[i]*z[i] for i in I2), GRB.MAXIMIZE)
    model.addConstrs((quicksum(A[i][j]*x[j] for j in J2) >= z[i] for i in I2))
    model.addConstr(quicksum(x[j] for j in J2) == p)

    model.update()
    model.optimize()

    return {j: x[j].X for j in J2}, {i: z[i].X for i in I2}



I2 = [i for i in range(10)]
J2 = [j for j in range(10)]

h2 = [18, 27, 26, 10, 11, 20, 14, 31, 35, 30]
A = [[1, 0, 0, 1, 1, 0, 0, 0, 0, 0],
     [0, 1, 0, 0, 0, 1, 0, 0, 0, 0],
     [0, 0, 1, 0, 0, 0, 0, 0, 0, 0],
     [1, 0, 0, 1, 0, 0, 0, 0, 0, 0],
     [1, 0, 0, 0, 1, 0, 0, 0, 0, 0],
     [0, 1, 0, 0, 0, 1, 0, 0, 1, 0],
     [0, 0, 0, 0, 0, 0, 1, 0, 0, 0],
     [0, 0, 0, 0, 0, 0, 0, 1, 1, 0],
     [0, 0, 0, 0, 0, 1, 0, 1, 1, 1],
     [0, 0, 0, 0, 0, 0, 0, 0, 1, 1]]

x, z = MCLP(h2, 1, A)
x, z

Gurobi Optimizer version 12.0.3 build v12.0.3rc0 (win64 - Windows 11.0 (26100.2))

CPU model: 13th Gen Intel(R) Core(TM) i7-13700, instruction set [SSE2|AVX|AVX2]
Thread count: 16 physical cores, 24 logical processors, using up to 24 threads

Optimize a model with 11 rows, 20 columns and 42 nonzeros
Model fingerprint: 0x10cccef1
Variable types: 0 continuous, 20 integer (20 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e+01, 4e+01]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+00]
Found heuristic solution: objective 65.0000000
Presolve removed 11 rows and 20 columns
Presolve time: 0.00s
Presolve: All rows and columns removed

Explored 0 nodes (0 simplex iterations) in 0.01 seconds (0.00 work units)
Thread count was 1 (of 24 available processors)

Solution count 2: 116 65 

Optimal solution found (tolerance 1.00e-04)
Best objective 1.160000000000e+02, best bound 1.160000000000e+02, gap 0.0000%


({0: 0.0,
  1: 0.0,
  2: 0.0,
  3: 0.0,
  4: 0.0,
  5: 0.0,
  6: 0.0,
  7: 0.0,
  8: 1.0,
  9: 0.0},
 {0: 0.0,
  1: 0.0,
  2: 0.0,
  3: 0.0,
  4: 0.0,
  5: 1.0,
  6: 0.0,
  7: 1.0,
  8: 1.0,
  9: 1.0})

For $P=1$, we get the same solution as the ADD Algorithm, with a facility at $I$, covering nodes $I, F, H, J$ and total demand covering 116.

## Problem 2c)

### IP Solver

In [ ]:
x2, z2 = MCLP(h2, 2, A)
x2, z2

Gurobi Optimizer version 12.0.3 build v12.0.3rc0 (win64 - Windows 11.0 (26100.2))

CPU model: 13th Gen Intel(R) Core(TM) i7-13700, instruction set [SSE2|AVX|AVX2]
Thread count: 16 physical cores, 24 logical processors, using up to 24 threads

Optimize a model with 11 rows, 20 columns and 42 nonzeros
Model fingerprint: 0x543dfa94
Variable types: 0 continuous, 20 integer (20 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e+01, 4e+01]
  Bounds range     [1e+00, 1e+00]
  RHS range        [2e+00, 2e+00]
Found heuristic solution: objective 93.0000000
Presolve removed 2 rows and 2 columns
Presolve time: 0.00s
Presolved: 9 rows, 18 columns, 38 nonzeros
Variable types: 0 continuous, 18 integer (18 binary)

Root relaxation: objective 1.550000e+02, 8 iterations, 0.01 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

*    0     0   

({0: 1.0,
  1: 0.0,
  2: -0.0,
  3: 0.0,
  4: 0.0,
  5: -0.0,
  6: -0.0,
  7: 0.0,
  8: 1.0,
  9: -0.0},
 {0: 1.0,
  1: 0.0,
  2: -0.0,
  3: 1.0,
  4: 1.0,
  5: 1.0,
  6: -0.0,
  7: 1.0,
  8: 1.0,
  9: 1.0})

Again, we get the same solution as before with facilities located at $A, I$, covering nodes $A, D, E, F, H, I, J$, and total demand covered of 155.

# Problem 3

In [ ]:
import pandas as pd
df = pd.read_excel('network_data.xlsx')
df.head()

demands = {}
for i in range(88):
    for l in range(5):
        demands[(i,l)] = float(df.iloc[i+1, l+1])

fixed_costs_j = []
for i in range(88):
    fixed_costs_j.append(float(df.iloc[i+92, 2]))
fixed_costs_k = [float(1000000) for k in range(10)]

cap_j = [float(2000) for j in range(88)]
cap_k = [float(10000) for k in range(10)]

t_costs_ji = {}
for i in range(88):
    for j in range(88):
        t_costs_ji[(j,i)] = float(df.iloc[i+184, j+1])

t_costs_kj = {}
for j in range(88):
    for k in range(10):
        t_costs_kj[(k,j)] = float(df.iloc[j+275, k+1])

In [ ]:
model = Model("CFLP")
I = [i for i in range(88)]
J = I
K = [k for k in range(10)]
L = [l for l in range(5)]

cities = model.addVars(I, vtype=GRB.BINARY, name="cities")
plants = model.addVars(K, vtype=GRB.BINARY, name="plants")
flow_kj = model.addVars(K, J, L, vtype=GRB.CONTINUOUS, lb=0, name="flow_plants_to_cities")
flow_ji = model.addVars(J, I, L, vtype=GRB.CONTINUOUS, lb=0, name="flow_cities_to_customers")

model.setObjective(
    quicksum(fixed_costs_j[j] * cities[j] for j in J) +
    quicksum(fixed_costs_k[k] * plants[k] for k in K) +
    0.25 * quicksum(t_costs_kj[(k, j)] * flow_kj[k, j, l] for k in K for j in J for l in L) +
    0.5 * quicksum(t_costs_ji[(j, i)] * flow_ji[j, i, l] for j in J for i in I for l in L),
    GRB.MINIMIZE)

model.addConstrs((quicksum(flow_ji[j, i, l] for j in J) == demands[(i, l)] for i in I for l in L), name="demand_satisfaction")
model.addConstrs((flow_ji[j, i, l] <= demands[(i, l)] * cities[j] for j in J for i in I for l in L), name="open_city")
model.addConstrs((flow_kj[k, j, l] <= sum(demands[(i, l)] for i in I) * plants[k] for k in K for j in J for l in L), name="open_plant")
model.addConstrs((quicksum(flow_kj[k, j, l] for k in K) == quicksum(flow_ji[j, i, l] for i in I) for j in J for l in L), name="flow_balance")
model.addConstrs((quicksum(flow_ji[j, i, l] for i in I for l in L) <= cap_j[j]*cities[j] for j in J), name="city_cap")
model.addConstrs((quicksum(flow_kj[k, j, l] for j in J for l in L) <= cap_k[k]*plants[k] for k in K), name="plant_cap")

model.update()
model.optimize()

Gurobi Optimizer version 12.0.3 build v12.0.3rc0 (win64 - Windows 11.0 (26100.2))

CPU model: 13th Gen Intel(R) Core(TM) i7-13700, instruction set [SSE2|AVX|AVX2]
Thread count: 16 physical cores, 24 logical processors, using up to 24 threads

Optimize a model with 44098 rows, 43218 columns and 211298 nonzeros
Model fingerprint: 0x15dfbb1d
Variable types: 43120 continuous, 98 integer (98 binary)
Coefficient statistics:
  Matrix range     [2e-01, 1e+04]
  Objective range  [3e+00, 1e+06]
  Bounds range     [1e+00, 1e+00]
  RHS range        [2e-01, 7e+02]
Found heuristic solution: objective 2.735653e+07
Presolve time: 0.13s
Presolved: 44098 rows, 43218 columns, 211298 nonzeros
Variable types: 43120 continuous, 98 integer (98 binary)
Deterministic concurrent LP optimizer: primal and dual simplex
Showing primal log only...

Concurrent spin time: 0.13s

Solved with primal simplex

Root relaxation: objective 3.065100e+06, 15437 iterations, 1.48 seconds (3.43 work units)

    Nodes    |    Curr

In [ ]:
print("Optimal cost:", model.ObjVal)

Optimal cost: 4680756.446863113


In [ ]:
for j in J:
    if cities[j].X > 0.5:
        print("Open city at location:", j+1, "with cost:", fixed_costs_j[j])
print("\nTotal cost of opened cities:", quicksum(fixed_costs_j[j] * cities[j].X for j in J).getValue())
print("")
for k in K:
    if plants[k].X > 0.5:
        print("Open plant at location:", k+1, "with cost:", fixed_costs_k[k])
print("\nTotal cost of opened plants:", quicksum(fixed_costs_k[k] * plants[k].X for k in K).getValue())

Open city at location: 3 with cost: 78700.0
Open city at location: 4 with cost: 58000.0
Open city at location: 5 with cost: 49400.0
Open city at location: 7 with cost: 25600.0
Open city at location: 9 with cost: 77100.0
Open city at location: 46 with cost: 80300.0
Open city at location: 67 with cost: 48800.0
Open city at location: 75 with cost: 38400.0
Open city at location: 84 with cost: 61500.0

Total cost of opened cities: 517800.0

Open plant at location: 6 with cost: 1000000.0
Open plant at location: 7 with cost: 1000000.0

Total cost of opened plants: 2000000.0


The optimal solution has us opening 2 plants at locations 6 and 7, each with a fixed cost of \$1,000,000, and open 9 facilities in cities 3, 4, 5, 7, 9, 46, 67, 75, and 84, with a total fixed cost of \$517,800. This gives a total fixed cost of \$2,517,800, meaning the total transportation cost is \$2,162,956.45.